In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize,scale,QuantileTransformer
from sklearn.metrics.pairwise import pairwise_distances
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import time
import pickle


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def stretch(x):
    return (x-x.min())/(x.max()-x.min())

In [2]:
df = pd.read_csv("../data/train.csv").drop("id",axis=1)
df_test = pd.read_csv("../data/test.csv").drop("id",axis=1)

labels = df["target"].values
print(type(labels))
print(labels.shape)

unique_labels = np.unique(labels)
nb_labels = len(unique_labels)

df_train = df.drop("target", axis=1)

X = df_train.values
X_test = df_test.values

print(X.shape)
print(X_test.shape)

y = []
for label in labels:
    y.append(int(label[-1:])-1)
y = np.asarray(y)


print("Construction des centroïdes")
centroids = []
for label in unique_labels:
    same_labels = df_train.loc[df['target'] == label]
    same_labels = same_labels.values
    centroids.append(same_labels.sum(axis = 0)/same_labels.shape[0])
centroids = np.asarray(centroids)


print("Construction de la représentation dcDistance")

X_dc = pairwise_distances(X,centroids)
X_test_dc = pairwise_distances(X_test, centroids, metric='minkowski')

print("Construction terminée \n")

y_train = y

<class 'numpy.ndarray'>
(61878,)
(61878, 93)
(144368, 93)
Construction des centroïdes
Construction de la représentation dcDistance
Construction terminée 



In [3]:
qt = QuantileTransformer(n_quantiles=10000, random_state=0)

In [4]:
res_test = qt.fit_transform(X_test_dc[:,0].reshape(-1,1))
res_train = qt.fit_transform(X_dc[:,0].reshape(-1,1))


for i in range(1,nb_labels):
    res_test_i = qt.fit_transform(X_test_dc[:,i].reshape(-1,1))
    res_test = np.concatenate((res_test,res_test_i), axis = 1)
    res_train_i = qt.fit_transform(X_dc[:,i].reshape(-1,1))
    res_train = np.concatenate((res_train,res_train_i), axis = 1)


In [5]:
print(res_train.shape)

(61878, 9)


In [7]:
# res_final = np.array(list(map(lambda x : (x-x.min())/(x.max()-x.min()+0.0001), res)))
# res_final = np.array(list(map(lambda x : np.round(x/x.sum(), decimals=2), res_final)))


In [52]:
start = time.time()

k_neighbors = 15


print("Apprentissage données en représentation dcDistance")


knn = KNeighborsClassifier(n_neighbors = k_neighbors)
knn.fit(res_train, y_train)


print('done')

Apprentissage données en représentation dcDistance
done


In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [15]:
mnb = MultinomialNB()
mnb.fit(res_train,y_train)

lgr = LogisticRegression(penalty='l1')
lgr.fit(res_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
pred_proba_mnb = mnb.predict_proba(res_test)
pred_proba_lgr = mnb.predict_proba(res_test)


pickle.dump(pred_proba_lgr, open('../data/dc.csv', 'wb'))



[0.02899799 0.27415256 0.13746532 0.04560339 0.0209019  0.22553455
 0.04783318 0.14064984 0.07886129]
